Read Mesh

In [13]:
from lnas import LnasFormat
import pathlib

meshes_path = pathlib.Path("./fixtures//lnas")

mesh = LnasFormat.from_file(meshes_path / "G100.lnas")
negative_mesh = LnasFormat.from_file(meshes_path / "G100_negative.lnas")

Offset body

In [14]:
import numpy as np

NORMAL_OFFSET = - 0.125


for surface_name in negative_mesh.surfaces.keys():
    tri_normals = negative_mesh.geometry_from_surface(surface_name).normals.copy()
    tri_verts = negative_mesh.geometry_from_surface(surface_name).triangle_vertices.copy()
    tri_verts += np.repeat(tri_normals, 3, axis=0).reshape(-1, 3, 3) * NORMAL_OFFSET

    new_verts = tri_verts.copy().reshape(-1, 3)
    new_verts = np.unique(new_verts, axis=0)

    new_triangles = []

    for tri in tri_verts:
        tri_index = []
        for v in tri:
            matches = np.all(new_verts == v, axis=1)
            indices = np.where(matches)
            tri_index.append(indices[0][0])
        tri_index.reverse()
        new_triangles.append(np.array(tri_index))

    new_triangles = np.array(new_triangles, dtype=np.uint32) + len(mesh.geometry.vertices)

    new_tri_indices = np.arange(
        mesh.geometry.triangles.shape[0], mesh.geometry.triangles.shape[0] + new_triangles.shape[0]
    )
    mesh.surfaces[surface_name + "_inside"] = new_tri_indices

    mesh.geometry.vertices = np.concatenate((mesh.geometry.vertices, new_verts))
    mesh.geometry.triangles = np.concatenate((mesh.geometry.triangles, new_triangles))

    mesh.geometry._update_triangles_vertices()
    mesh.geometry._update_normals()
    mesh.geometry._update_areas()
    mesh.geometry._update_vertices_normals()


filename = meshes_path / "G100.merged.lnas"
export_path = meshes_path / "G100.merged.stl"

mesh.to_file(filename)
geometry = mesh.geometry_from_surface("m_000")
# geometry.export_stl(meshes_path / "m_000_inside.stl")
print(geometry.vertices[:, 0].min(), geometry.vertices[:, 0].max())
print(geometry.vertices[:, 1].min(), geometry.vertices[:, 1].max())
print(geometry.vertices[:, 2].min(), geometry.vertices[:, 2].max())
# mesh.export_stl(export_path)

-427.4785 -124.554756
-245.32094 159.66632
837.60004 854.25


In [15]:
mesh = LnasFormat.from_file(meshes_path / "G100.merged.lnas")

# mesh.geometry_from_surface("m_000_inside").export_stl(meshes_path / "m_000_inside.stl")
geometry = mesh.geometry_from_surface("m_000_inside")
print(geometry.vertices[:, 0].min(), geometry.vertices[:, 0].max())
print(geometry.vertices[:, 1].min(), geometry.vertices[:, 1].max())
print(geometry.vertices[:, 2].min(), geometry.vertices[:, 2].max())

-427.4785 -124.554756
-245.32094 159.66632
837.60004 854.25
